In [1]:
from os import chdir  # 작업 위치를 바꾸기 위한 헤드
chdir('C:\\Users\\ells2\\Documents\\project') # 작업 위치 변경

import sys,os
sys.path.append(os.pardir)

In [2]:
import pandas as pd
import numpy as np

In [105]:
review = pd.read_csv("project_review.csv",encoding='cp949')

In [106]:
score = pd.read_csv("project_score.csv",encoding='cp949')
del score["개봉년도"]
del score["개봉월"]
del score["개봉일"]
del score["런타임"]

# 리뷰 인덱스 맞추기

In [107]:
score = score.sort_values(by="title")
score = score.reset_index()
del score['index']

In [108]:
score['등급'] = score['등급'].fillna("전체")

# 스코어 전처리

In [110]:
def replacing(m):
    m = m.replace("[","")
    m = m.replace("'","")
    m = m.replace("]","")
    s = m.split(',')
    a = [t.strip() for t in s]
    return a

In [116]:
def prepro(score):
    genre = score["장르"]
    genre_re = [replacing(r) for r in genre]
    genre_unique = np.unique([b for a in genre_re for b in a])[1:]
    result = []
    for b in genre_re:
        d=[]
        for a in genre_unique:
            d.append(int(a in b))
        result.append(d)
    genre_dummy = pd.DataFrame(result,columns=genre_unique)
    #
    age_dummy = pd.get_dummies(score["등급"],drop_first=True)
    
    return(pd.concat([score,genre_dummy,age_dummy],axis=1))

In [118]:
score_pre = prepro(score)

In [120]:
del score_pre['장르']
del score_pre['등급']

# 리뷰 변수화

## 필요없는거 빼기

In [11]:
def tagged_to_list(a):
    a = a.replace('\'','')
    a = a.replace('[','')
    a = a.replace(']','')
    a = a.replace(' ','')
    return a

In [ ]:
def stopwords(word,stops):
    for i in range(len(stops)):
        if stops[i] in word:
            word = word.replace(stops[i],'m')
    return(word)

In [ ]:
stops = ['최승현','차승원','소지섭','서인국','박서준','이선균','신세경','박재범','류승룡','재범','유승호','재범','윤계상',
'손현주','여진구','심은경','송혜교','설리','박유천','최민식','조진웅','성동일','최강희','김혜수','봉준호','이제훈',
'이민호','임창정','오달수','신하균','이준기','김래원','이정재','송중기','정재영','조승우','손예진','임시완','한석규','김남길','클라라',
'류준열','박보영','조정석','김명민','박해일','유아인','하지원','김우빈','정우성','차태현','현빈','김윤석','김수현','도경수',
'이종석','황정민','설경구','심형래','이병헌','마동석','하정우','강동원','한효주']

In [34]:
def pos_pop(sentence):
    temp = []
    for i in range(len(sentence)):
        if 'Josa' in sentence[i]:
            continue
        if 'Punctuation' in sentence[i]:
            continue
        if 'Alpha' in sentence[i]:
            continue
        if 'Determiner' in sentence[i]:
            continue
        if 'Suffix' in sentence[i]:
            continue
        if 'Eomi' in sentence[i]:
            continue
        if len(sentence[i].split('/')[0]) < 2:
            continue
        temp.append(sentence[i])
    return(temp)

In [ ]:
pos_list_st = [stopwords(a,stops) for a in pos_list]
pos_list_s = [b.split(',') for b in pos_list_st]
postagged_pp = [pos_pop(s) for s in pos_list_s]

In [35]:
postagged_pop = [pos_pop(s) for s in pos_list_s]

In [36]:
review['pop'] = postagged_pop

## 키워드 세기

In [37]:
movie_name = np.unique(review['id'])

In [58]:
def each_movie_word(review,movie_name):
    temp_movie = review[review['id']==movie_name]['pop']
    keyword = Counter([i for item in temp_movie for i in item])
    return keyword

In [93]:
from collections import Counter
import operator
#전체 키워드 모으기
keyword_cnt = Counter([i for item in review['pop'] for i in item])
keyword_clip = sorted(keyword_cnt.items(), key=operator.itemgetter(1))[-1000:]

##  리뷰 변수화

In [95]:
movie_coef = []
for i in range(len(movie_name)):
    movie_count = each_movie_word(review,movie_name[i])
    movie_count_keys = list(movie_count.keys())
    temp = []
    for j in range(len(keyword_clip)):
        if keyword_clip[j][0] in movie_count_keys:
            temp.append(movie_count[keyword_clip[j][0]])
        else:
            temp.append(0)
    movie_coef.append(temp)

In [96]:
keyword_count = list(dict(keyword_clip).values())
keyword_name = dict(keyword_clip).keys()

In [97]:
movie_word_count = pd.DataFrame(movie_coef)
movie_word_count.columns = keyword_name
movie_word_count['movie_list'] = movie_name

In [98]:
all_movie_df = pd.DataFrame(np.array(movie_coef) / np.array(keyword_count))
all_movie_df.columns = keyword_name
all_movie_df['movie_list'] = movie_name

## 합치기

In [121]:
score_fin = pd.concat([score_pre,all_movie_df],axis=1)

In [122]:
del score_fin['movie_list']

score_fin.to_csv('dasolabutaghe.csv')

 #  분석

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [125]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [129]:
f1s = []
acc = []
recall = []
prec = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(score_fin.drop(["y","title"],axis=1),score_fin["y"],test_size=0.1)
    linear = LogisticRegression()
    linear.fit(X_train,y_train)
    acc.append(linear.score(X_test,y_test))
    f1s.append(f1_score(y_test,linear.predict(X_test)))
    recall.append(recall_score(y_test,linear.predict(X_test)))
    prec.append(precision_score(y_test,linear.predict(X_test)))

In [130]:
pd.DataFrame([f1s ,acc , recall,prec],index=['f1','acc','recall','precision'])

,0,1,2,3,4,5,6,7,8,9
f1,0.523810,0.421053,0.418605,0.242424,0.465116,0.400000,0.347826,0.222222,0.409091,0.222222
acc,0.881657,0.869822,0.852071,0.852071,0.863905,0.893491,0.822485,0.834320,0.846154,0.834320
recall,0.423077,0.285714,0.290323,0.160000,0.344828,0.260870,0.242424,0.137931,0.281250,0.129032
precision,0.687500,0.800000,0.750000,0.500000,0.714286,0.857143,0.615385,0.571429,0.750000,0.800000


In [132]:
np.mean(f1s)

0.36723687685371476

In [133]:
np.mean(acc)

0.85502958579881638

In [134]:
np.mean(recall)

0.25554484758321749

In [135]:
np.mean(prec)

0.70457417582417581